In [3]:
%pip install datasets
%pip install transformers
%pip install peft
%pip install evaluate
%pip install scikit-learn



from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)
from turtle import pd
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np
import pandas as pd

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


c:\Users\zhuocheng\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [61]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('Joanne/Metaphors_and_Analogies','Pairs_Jankowiac_random_split')
dataset







Generating train split: 180 examples [00:00, 22576.32 examples/s]

Generating validation split: 36 examples [00:00, 4009.64 examples/s]

Generating test split: 144 examples [00:00, 19575.41 examples/s]


DatasetDict({
    train: Dataset({
        features: ['corpus', 'id', 'set_id', 'label', 'sentence', 'A', 'B', 'A_position', 'B_position', '5-folds'],
        num_rows: 180
    })
    validation: Dataset({
        features: ['corpus', 'id', 'set_id', 'label', 'sentence', 'A', 'B', 'A_position', 'B_position', '5-folds'],
        num_rows: 36
    })
    test: Dataset({
        features: ['corpus', 'id', 'set_id', 'label', 'sentence', 'A', 'B', 'A_position', 'B_position', '5-folds'],
        num_rows: 144
    })
})

In [65]:
# Get the first row of the dataset
first_row = dataset['train']

# Print the first row
print(first_row)

Dataset({
    features: ['corpus', 'id', 'set_id', 'label', 'sentence', 'A', 'B', 'A_position', 'B_position', '5-folds'],
    num_rows: 180
})


In [66]:
# assume 'dataset' is your dataset object
label_array = np.array(dataset['train']['label'])

label_dtype = label_array.dtype

label_array

array([2, 1, 2, 1, 1, 0, 0, 2, 1, 1, 2, 0, 1, 2, 0, 0, 1, 0, 2, 0, 2, 2,
       2, 1, 0, 0, 0, 0, 2, 2, 0, 1, 0, 1, 2, 0, 0, 1, 1, 1, 0, 1, 0, 2,
       0, 1, 0, 0, 0, 1, 0, 2, 2, 0, 0, 0, 0, 2, 0, 2, 1, 2, 1, 1, 2, 2,
       1, 0, 1, 1, 0, 2, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 2, 2, 2, 0, 0, 0,
       0, 2, 0, 1, 2, 2, 1, 0, 1, 0, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 0, 1, 1, 1, 1, 2, 0, 0, 1, 1, 0, 2,
       2, 1, 1, 2, 2, 1, 0, 2, 2, 1, 0, 1, 1, 0, 2, 1, 0, 1, 0, 0, 0, 1,
       0, 2, 1, 0, 2, 0, 1, 0, 0, 1, 2, 1, 2, 0, 0, 0, 1, 0, 0, 2, 2, 0,
       2, 0, 0, 2])

In [67]:
model_checkpoint = 'distilbert-base-uncased'
# define label maps
id2label = {0: "Anomaly", 1: "literal", 2 : "metaphor"}
label2id = {"Anomaly":0, "literal":1, "metaphor":2}

# generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=3, id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [68]:
# create tokenize function

# create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))


def tokenize_function(examples):
    # extract text
    text = examples["sentence"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs


# tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset


Map: 100%|██████████| 180/180 [00:00<00:00, 22574.29 examples/s]

Map: 100%|██████████| 36/36 [00:00<00:00, 6020.29 examples/s]

Map: 100%|██████████| 144/144 [00:00<00:00, 18026.02 examples/s]


DatasetDict({
    train: Dataset({
        features: ['corpus', 'id', 'set_id', 'label', 'sentence', 'A', 'B', 'A_position', 'B_position', '5-folds', 'input_ids', 'attention_mask'],
        num_rows: 180
    })
    validation: Dataset({
        features: ['corpus', 'id', 'set_id', 'label', 'sentence', 'A', 'B', 'A_position', 'B_position', '5-folds', 'input_ids', 'attention_mask'],
        num_rows: 36
    })
    test: Dataset({
        features: ['corpus', 'id', 'set_id', 'label', 'sentence', 'A', 'B', 'A_position', 'B_position', '5-folds', 'input_ids', 'attention_mask'],
        num_rows: 144
    })
})

In [69]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [70]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

# define an evaluation function to pass into trainer later
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

In [71]:
# define list of examples
text_list = ["It was good.", "Not a fan, don't recommed.", "Better than the first one.", "This is not worth watching even once.", "This one is a pass."]

print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)

    print(text + " - " + id2label[predictions.tolist()])

Untrained model predictions:
----------------------------
It was good. - literal
Not a fan, don't recommed. - literal
Better than the first one. - literal
This is not worth watching even once. - literal
This one is a pass. - literal


In [72]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['q_lin'])

In [73]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, r=4, target_modules={'q_lin'}, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [74]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 629,763 || all params: 67,585,542 || trainable%: 0.9318013607111414


In [75]:
# hyperparameters
lr = 1e-3
batch_size = 4
num_epochs = 10

In [76]:
# define training arguments
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [77]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics,
)

# train model
trainer.train()

 10%|█         | 45/450 [00:03<00:28, 14.26it/s]

{'eval_loss': 0.4888295531272888, 'eval_accuracy': {'accuracy': 0.8055555555555556}, 'eval_runtime': 0.2714, 'eval_samples_per_second': 132.647, 'eval_steps_per_second': 33.162, 'epoch': 1.0}


 20%|██        | 90/450 [00:06<00:22, 16.15it/s]

{'eval_loss': 0.5225620269775391, 'eval_accuracy': {'accuracy': 0.8055555555555556}, 'eval_runtime': 0.2578, 'eval_samples_per_second': 139.621, 'eval_steps_per_second': 34.905, 'epoch': 2.0}


 30%|███       | 135/450 [00:10<00:18, 16.99it/s]

...

100%|██████████| 450/450 [00:35<00:00, 16.06it/s]

{'eval_loss': 1.009372353553772, 'eval_accuracy': {'accuracy': 0.75}, 'eval_runtime': 0.2581, 'eval_samples_per_second': 139.498, 'eval_steps_per_second': 34.874, 'epoch': 10.0}


100%|██████████| 450/450 [00:35<00:00, 12.56it/s]

{'train_runtime': 35.835, 'train_samples_per_second': 50.23, 'train_steps_per_second': 12.558, 'train_loss': 0.20067691379123265, 'epoch': 10.0}


TrainOutput(global_step=450, training_loss=0.20067691379123265, metrics={'train_runtime': 35.835, 'train_samples_per_second': 50.23, 'train_steps_per_second': 12.558, 'total_flos': 3656219502240.0, 'train_loss': 0.20067691379123265, 'epoch': 10.0})